In [124]:
# HYPOTHESYS
# We expect that mentions of Trumps will have 50/50 positive and negative
# We expect more positive tweets volume from republican states
# We expect Trump's tweets to be majority negative
# What are the main keywords Trump uses and what are the sentiment about those subjects

In [125]:
# import dependencies 
import os
import pandas as pd
from matplotlib import pyplot as plt
import time
import csv
import json
import requests
from config import api_key, api_secret_key
import gmaps
import re 
import twitter
import tweepy 
from tweepy import Stream
from tweepy import OAuthHandler
from tweepy.streaming import StreamListener
from textblob import TextBlob 
import base64

# Access maps with unique API key
# gmaps.configure(api_key=gkey)

# url for API

# url = 'https://api.twitter.com/1.1/search/tweets.json'

# # Store a the twitter handle mentions we will use

# handle = '@realDonaldTrump'

# Filter the request for  "geo_enabled": true, language to be only english (lang=en), restrict by region = United Sates
# Reference the url to request the api data, but not going over the 
# request limit per second (max_number_of_requests = 180 per 15 min)
# https://towardsdatascience.com/creating-the-twitter-sentiment-analysis-program-in-python-with-naive-bayes-classification-672e5589a7ed

# convert to JSON object 

# Create a dictionary with the data we need:
            
# print the request to get an idea of the set

# Reference the url for the sentiment analysis code 'https://www.geeksforgeeks.org/twitter-sentiment-analysis-using-python/''



In [126]:
# Authenticate and Make the first API request

client_key = api_key
client_secret = api_secret_key

key_secret = '{}:{}'.format(client_key, client_secret).encode('ascii')
b64_encoded_key = base64.b64encode(key_secret)
b64_encoded_key = b64_encoded_key.decode('ascii')

base_url = 'https://api.twitter.com/'
auth_url = '{}oauth2/token'.format(base_url)

auth_headers = {
    'Authorization': 'Basic {}'.format(b64_encoded_key),
    'Content-Type': 'application/x-www-form-urlencoded;charset=UTF-8'
}

auth_data = {
    'grant_type': 'client_credentials'
}

auth_resp = requests.post(auth_url, headers=auth_headers, data=auth_data)

auth_resp.json().keys()

access_token = auth_resp.json()['access_token']

search_headers = {
    'Authorization': 'Bearer {}'.format(access_token)    
}




In [137]:
# Adjust parameters for test run
from collections import defaultdict 
search_params = {
    'q': '@realDonaldTrump',
    'result_type': 'recent',
    'count': 10
}

# Date(created_at), Location (geocode), , Tweet (text), (user_mentions),  (hashtags),
#(symbols), user id ("user": id, id_string)
# followers volume (followers_count), friends count (friends_count), (favourites_count)
# (retweet_count), (retweeted), (possibly_sensitive), 


search_url = '{}1.1/search/tweets.json'.format(base_url)

search_resp = requests.get(search_url, headers=search_headers, params=search_params)

tweet_data = search_resp.json()

tweet_data_filter = defaultdict(list)

# print(json.dumps(tweet_data, indent=4))


# print(tweet_data['statuses'][0]['text'])


statuses = tweet_data['statuses']
# l1 = []
# l2 = []

for status in statuses:
    
#     l1.append(status['text'])
#     l2.append(status['created_at'])

    tweet_data_filter["text"].append(status['text'])
    tweet_data_filter["created_at"].append(status['created_at'])
    
# tweet_data_filter = {}    
# tweet_data_filter["text"] = l1
# tweet_data_filter["created_at"] = l2

# print(type(tweet_data_filter["text"]))
# print(type(tweet_data_filter["created_at"]))


tweet_data_filter_pdf = pd.DataFrame(tweet_data_filter)

tweet_data_filter_pdf




,text,created_at
0,RT @RepSwalwell: Two and a half years into his...,Wed Jun 26 04:33:07 +0000 2019
1,After Twitter killing my account I had since J...,Wed Jun 26 04:33:07 +0000 2019
2,@tburages @realDonaldTrump Big guy is close to...,Wed Jun 26 04:33:07 +0000 2019
3,RT @realDonaldTrump: ....Death to America. I t...,Wed Jun 26 04:33:07 +0000 2019
4,"RT @DonaldJTrumpJr: Good, when you’re dealing ...",Wed Jun 26 04:33:07 +0000 2019
5,RT @realDonaldTrump: https://t.co/JDS4zUXXJG,Wed Jun 26 04:33:07 +0000 2019
6,@realDonaldTrump Thoughts and prayers.,Wed Jun 26 04:33:07 +0000 2019
7,RT @chipfranklin: He's a fucking sociopath. @r...,Wed Jun 26 04:33:06 +0000 2019
8,@RyanHillMI @realDonaldTrump Collusion and obs...,Wed Jun 26 04:33:06 +0000 2019


In [ ]:
# DATA CLEANING

In [ ]:
# VISUALIZATION REQUEIREMENTS - 6-8

# What is the distribution of sentiment (positive/negative/neutral) per mention of handle?
# Piechart/bar chart per handle mention

# How many (positive/negative/neutral) retweets?

# Distribution of sentiment by state and comparison to state being democrat/republican
# Gmaps with pins


# Scatter plot - size of bubles is number of followers

# Most commom used words by Trump - top keywords
# Use of exclamation points
# Use of Caps lock?
# Emoji symbols he uses
# Sentiment analysis of Trump Tweets
# Sentiment analysis of his tweets for top 5 keyqords he uses (sort and select top 5 based on total count)


# Scatter plot of Trumps tweets colored by sentiment

# Save png versions of our visuals


In [ ]:
# OBSERVATIONS/TRENDS

# Implications - can we predict if he will be elected or not